# Understanding Zarr

In [62]:
import datetime
import json

from pathlib import Path
from typing import Any

import folium
import fsspec
import numpy as np
import planetary_computer
import pystac_client
import pyproj
import shapely

# Point of Interest (POI)

I want to go to Puerto Rico, but I'm from a northern temporate latitude. I'd like to find the best week of the year for me to visit Puerto Rico without having oppressive weather. Can we use a zarr dataset from Planetary Computer to answer this question?

To pick a more specific point of interest, let's use these coordinates of San Juan:

In [63]:
POI = shapely.Point(-66.063889, 18.406389)

And let's throw this point on a map, just so we're all aware of where we're looking for data.

In [64]:
location = (POI.coords[0][1], POI.coords[0][0])

point_map = folium.Map(
    location=location,
    tiles='CartoDB positron',
)

folium.CircleMarker(
    location=location,
    fill=True,
    fill_opacity=0.6,
).add_to(point_map)

point_map

## Finding data

Now that we know where to look, we need to find some data we can use to answer this question. Turns out Microsoft's Plantary Computer has zarr data, and [we can search for zarr to find possible datasets](https://planetarycomputer.microsoft.com/catalog?filter=zarr). A prime contender of the options is the [Daymet Daily Puerto Rico](https://planetarycomputer.microsoft.com/dataset/daymet-daily-pr) dataset. For the sake of keeping this exercise simple, let's just consider measurements from 2020 (the last year in the dataset), and we'll sum the daily maximum temperature (`tmax` variable).

Zarr datasets can be represented by a STAC collection without items, with a collection asset pointing to the zarr root. To get more information about this dataset we can fetch the `daymet-daily-pr` STAC collection from Planetary Computer. Pay special attention to the `assets`, `cube:variables`, and `cube:dimensions` attributes of the fetched collection.

Note that accessing Planetary Computer data from Azure Blob Storage requires a signed token; we can use the `planetary_computer` package as a "plugin" of sorts to `pystac_client` to ensure we get an access token attached as necessary to each asset.

In [65]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
collection = catalog.get_collection('daymet-daily-pr')
collection

<CollectionClient id=daymet-daily-pr>

## Accessing the zarr files

We're going to use `fsspec` with the `abfs` filesystem type (as provided by the `adlfs` package) to get access to the zarr directory tree and its files. The `abfs` filesystem requires both an access token signed to and the Azure account name of the bucket we are attempting to access. Because we used the `planetary_computer` package with `pystac_client` when querying the collection, these connection details are included in the `xarray:storage_options` property of the `zarr-abfs` asset, and we can use them to successfully initialize our filessytem object `fs` and use it to explore the contents of the bucket.

All the paths we're interested in will be under the root of the zarr we are interested in. That path is provided within the `zarr-abfs` asset's href.

In [66]:
asset = collection.assets["zarr-abfs"]
asset

<Asset href=abfs://daymet-zarr/daily/pr.zarr>

In [67]:
# we need the href without the `abfs://`
zarr_root = Path(asset.href.split('//', 1)[1])
zarr_root

PosixPath('daymet-zarr/daily/pr.zarr')

In [68]:
# the xarray:storage_options gives us the access token and account name required to connect
fs = fsspec.filesystem('abfs', **asset.extra_fields['xarray:storage_options'])

An fsspec filesystem provides an API that includes normal filesystem-related functions, such as listing a path (`ls`), reading files (`open), or finding file information (`stat`).

In [69]:
# now we can list the zarr root
fs.ls(str(zarr_root))

['daymet-zarr/daily/pr.zarr/.zattrs',
 'daymet-zarr/daily/pr.zarr/.zgroup',
 'daymet-zarr/daily/pr.zarr/.zmetadata',
 'daymet-zarr/daily/pr.zarr/dayl',
 'daymet-zarr/daily/pr.zarr/lambert_conformal_conic',
 'daymet-zarr/daily/pr.zarr/lat',
 'daymet-zarr/daily/pr.zarr/lon',
 'daymet-zarr/daily/pr.zarr/prcp',
 'daymet-zarr/daily/pr.zarr/srad',
 'daymet-zarr/daily/pr.zarr/swe',
 'daymet-zarr/daily/pr.zarr/time',
 'daymet-zarr/daily/pr.zarr/time_bnds',
 'daymet-zarr/daily/pr.zarr/tmax',
 'daymet-zarr/daily/pr.zarr/tmin',
 'daymet-zarr/daily/pr.zarr/vp',
 'daymet-zarr/daily/pr.zarr/x',
 'daymet-zarr/daily/pr.zarr/y',
 'daymet-zarr/daily/pr.zarr/yearday']

Note that when we list the zarr root we see some `.z*` files with zarr-related metadata and othe such info, which can be used to understand how to access the data in the listed directories. Notice also that the listed directory names predominately map to the zarr variables listed out in the STAC collection. From this we can be reasonably certain that the data for each varaible is nested within the directory of the same name.

In [70]:
# and we can read a file
with fs.open(str(zarr_root / '.zattrs')) as f:
   cnt = f.read()

cnt

b'{\n    "Conventions": "CF-1.6",\n    "Version_data": "Daymet Data Version 4.0",\n    "Version_software": "Daymet Software Version 4.0",\n    "citation": "Please see http://daymet.ornl.gov/ for current Daymet data citation information",\n    "references": "Please see http://daymet.ornl.gov/ for current information on Daymet references",\n    "source": "Daymet Software Version 4.0",\n    "start_year": 1980\n}'

## Making browsing even easier

Even with fsspec, browsing and reading files has a bit more boiler plate than we might like. We can easily create some simple functions to make an easier API for the types of operations we need to do to explore our zarr.

In [71]:
# let's make some convenience functions to make browsing easier
def ls_zarr(path: str) -> list[str]:
    return fs.ls(str(zarr_root / path))

def read_zarr_file(path: str) -> bytes:
    with fs.open(str(zarr_root / path)) as f:
        return f.read()

def read_zarr_json(path: str) -> dict[str, Any]:
    return json.loads(read_zarr_file(path))

def print_json(_json: dict[str, Any]) -> None:
    print(json.dumps(_json, indent=4))

And we can use one of the convenience functions to show much simpler reading the zarr metadata can be.

In [72]:
zmeta = read_zarr_json('.zmetadata')
print_json(zmeta)

{
    "metadata": {
        ".zattrs": {
            "Conventions": "CF-1.6",
            "Version_data": "Daymet Data Version 4.0",
            "Version_software": "Daymet Software Version 4.0",
            "citation": "Please see http://daymet.ornl.gov/ for current Daymet data citation information",
            "references": "Please see http://daymet.ornl.gov/ for current information on Daymet references",
            "source": "Daymet Software Version 4.0",
            "start_year": 1980
        },
        ".zgroup": {
            "zarr_format": 2
        },
        "dayl/.zarray": {
            "chunks": [
                365,
                231,
                364
            ],
            "compressor": {
                "blocksize": 0,
                "clevel": 5,
                "cname": "lz4",
                "id": "blosc",
                "shuffle": 1
            },
            "dtype": "<f4",
            "fill_value": "NaN",
            "filters": null,
            "order"

### `.zmetadata`

Notice that the `.zmetadata` file starts with the `.zattrs` key, the value of which contains the same content as the `.zattrs` file we read a moment ago? Spoiler alert: `.zmetadata` contains the contents of all the `.z*` files. It's a great way to get an overview of the entire contents of the zarr. From here we can see a full inventory of the `.z*` file types through the archive:

* `.zmetadata`: this file only
* `.zattrs`: top-level and one in each variable's directory
* `.zgroup`: top-level, just specifies the zarr version (here version 2)
* `.zarray`: one in each variable's directory

Note that `.zmetadata` appears only to be present if the zarr has "consolidated metadata", as this one does (and apparently that is a zarr extension and not part of the core spec).

### `.zattrs`

`.zattrs` appears to provide user-facing information, such as dataset version information, citations, and that which can help users appropriately interpret the meaning of the data including units, array dimensions, and the like. Note that the collection also contains much of this information. For our purposes these values can help us know which files we might want to look at and how to interpret their structure and values, but otherwise we will be programmatically skipping over them.

### `.zgroup`

This is super relevant to zarr readers to know what format the zarr is. For us, we are just exploring whatever we've found here, so it's not relevant really at all. At least not until we get to trying to read the next zarr file and we start wondering why it looks different than this one...

### `.zarray`

Finally, a good one. We're going to need to be very interested in the `.zarray` files because they tell us some key information we're going to need to properly read the data arrays. Specifically, the fields data type, compression type (note that all are compressed via the `blosc` algorithm with the same settings), and chunk size are ones we're absolutely going to need. Some other fields that could be very relevant for reading data in other files include `filters` and `fill_value`, but we won't need to be concerned with those here.

## Reading an array

We're going to need location information, and our POI is in WGS84 (lat/long), so maybe we start by reading the `lat` variable?

First, let's see what all is in the `lat` directory:

In [73]:
ls_zarr('lat/')

['daymet-zarr/daily/pr.zarr/lat/.zarray',
 'daymet-zarr/daily/pr.zarr/lat/.zattrs',
 'daymet-zarr/daily/pr.zarr/lat/0.0']

Not a lot. Maybe if we look at the `.zarray` and `.zattrs` we can learn something?

In [74]:
lat_zarray = read_zarr_json('lat/.zarray')
print_json(lat_zarray)

{
    "chunks": [
        231,
        364
    ],
    "compressor": {
        "blocksize": 0,
        "clevel": 5,
        "cname": "lz4",
        "id": "blosc",
        "shuffle": 1
    },
    "dtype": "<f4",
    "fill_value": "NaN",
    "filters": null,
    "order": "C",
    "shape": [
        231,
        364
    ],
    "zarr_format": 2
}


In [75]:
lat_zattrs = read_zarr_json('lat/.zattrs')
print_json(lat_zattrs)

{
    "_ARRAY_DIMENSIONS": [
        "y",
        "x"
    ],
    "long_name": "latitude coordinate",
    "standard_name": "latitude",
    "units": "degrees_north"
}


The contents of both of these are exactly what we saw in top-level `.zmetadata`. No suprise there. And the `.zattrs` contents aren't particularly helpful at the moment.

But the `.zarray`, that explains some things. First, we see the chunk size (`chunks`) is equal to the overall array size (`shape`). This tells us that this particular variable has only one chunk covering the entire extent of the data set to which it is relevant. We can make an educated guess then that `lat/0.0` is the array data for the single chunk.

Let's read it and see what it looks like.

In [76]:
lat_bytes = read_zarr_file('lat/0.0')
print(f'{lat_bytes[:100]}...')
print(len(lat_bytes))

b'\x02\x01!\x04\xd0!\x05\x00\x00\x00\x04\x00\xf8\x07\x02\x00\x18\x00\x00\x00\xff\x96\x01\x00\xe5\xfc\x00\x00\xf3\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xffv\xa9\xc6\xe2\xfd\x193Nh\x81\x9b\xb3\xcc\xe4\xfb\x13*@Vl\x81\x96\xaa\xbe\xd2\xe5\xf8\n\x1c.?Paq\x80\x90\x9e\xad\xbb\xc9\xd6\xe3\xef\xfb\x07\x12\x1d(2<ENW_'...
133112


Just a bunch of ugly binary data--as we might have suspected, espcially given that it is supposedly compressed.

Looking into the `blosc` compression format appears to indicate that it is `lz4`, so it seems like an `lz4`-compatible codec would be required. It turns out that's not entirely true. To cut a long story short, digging into this and trying to unravel what something like xarray uses to decompress such an array leads to [the `numcodecs` package](https://github.com/zarr-developers/numcodecs). It has a `blosc` codec, which can use to decompress this byte string and get an array.

We'll be doing that operation a bunch, so let's make another convenience function that we can use to quickly and easily read these compressed array files.

In [77]:
import numcodecs.blosc

def read_zarr_blosc(path: str) -> Any:
    #with lz4.frame.LZ4FrameDecompressor() as decompressor:
    return numcodecs.blosc.decompress(read_zarr_file(path))

In [78]:
lat_uncompressed = read_zarr_blosc('lat/0.0')
print(f'{lat_uncompressed[:100]}...')
print(len(lat_uncompressed))

b'\xa9u\x9fA\xc6n\x9fA\xe2g\x9fA\xfd`\x9fA\x19Z\x9fA3S\x9fANL\x9fAhE\x9fA\x81>\x9fA\x9b7\x9fA\xb30\x9fA\xcc)\x9fA\xe4"\x9fA\xfb\x1b\x9fA\x13\x15\x9fA*\x0e\x9fA@\x07\x9fAV\x00\x9fAl\xf9\x9eA\x81\xf2\x9eA\x96\xeb\x9eA\xaa\xe4\x9eA\xbe\xdd\x9eA\xd2\xd6\x9eA\xe5\xcf\x9eA'...
336336


Uncompressed it's still pretty ugly, but again not really unexpected because it's binary data. We need to unpack it into an array and see what that looks like.

#### A quick note on data types

The format of the data type is not what we might suspect after all the work we did with the `struct` package and packing/unpacking the TIFF values. Apparently the data type specification here is one natively supported by `numpy`, meaning the `dtype` string can be interpreted directly as a data type by `numpy`:

In [79]:
np.dtype('<f4'), np.dtype('<f8'), np.dtype('>f2')

(dtype('float32'), dtype('float64'), dtype('>f2'))

`numpy` supports unpacking binary data from a byte string natively via the `frombuffer` method. We merely need to provide the specified data type for it to unpack the bytes properly, and then we can reshape the resulting array into the `chunks` shape.

In [80]:
dt = np.dtype(lat_zarray['dtype'])
lat_array = np.frombuffer(lat_uncompressed, dtype=dt).reshape(lat_zarray['chunks'])
lat_array

array([[19.932451, 19.929089, 19.925724, ..., 18.66994 , 18.666313,
        18.662685],
       [19.924347, 19.920984, 19.91762 , ..., 18.66203 , 18.658403,
        18.654774],
       [19.916243, 19.91288 , 19.909517, ..., 18.654118, 18.650492,
        18.646864],
       ...,
       [18.091873, 18.088625, 18.085377, ..., 16.872786, 16.869282,
        16.865776],
       [18.083834, 18.080587, 18.07734 , ..., 16.864933, 16.861431,
        16.857927],
       [18.075796, 18.072548, 18.069302, ..., 16.857082, 16.85358 ,
        16.850077]], dtype=float32)

Success! Those values should look like what we'd expect for latitude values in or around Puerto Rico.

We can read in the `lon` variable the exact same way:

In [81]:
lon_zarray = read_zarr_json('lon/.zarray')
dt = np.dtype(lon_zarray['dtype'])
lon_array = np.frombuffer(read_zarr_blosc('lon/0.0'), dtype=dt).reshape(lon_zarray['chunks'])
lon_array

array([[-67.1878  , -67.17923 , -67.170654, ..., -64.14222 , -64.13392 ,
        -64.12562 ],
       [-67.19135 , -67.182785, -67.17422 , ..., -64.14603 , -64.137726,
        -64.129425],
       [-67.19491 , -67.18634 , -67.17777 , ..., -64.14983 , -64.14153 ,
        -64.13323 ],
       ...,
       [-67.98011 , -67.9717  , -67.963295, ..., -64.99068 , -64.98253 ,
        -64.97437 ],
       [-67.983505, -67.9751  , -67.96669 , ..., -64.994316, -64.98617 ,
        -64.97801 ],
       [-67.98689 , -67.97849 , -67.970085, ..., -64.997955, -64.9898  ,
        -64.98165 ]], dtype=float32)

## Finding the cell coordinates of our POI

Now that we have the `lat` and `lon` variable arrays, we should be able to locate the cell coordinates corresponding to our POI, right?

Well, if we look closely at the arrays we realize something: they are two-dimensional! Neither latitude nor longitude are actually one of the base dimensions of our data cube. If we look more closely at the collection's metadata we realize that the data are in a projected coordinate system. We _could_ presumably use the `lat` and `long` variable arrays to find our cell coordinates, but doing so would require some sort of non-obvious multivariate optimization problem.

Instead, what happens if we open our `x` variable?

In [82]:
x_zarray = read_zarr_json('x/.zarray')
dt = np.dtype(x_zarray['dtype'])
x_array = np.frombuffer(read_zarr_blosc('x/0'), dtype=dt).reshape(x_zarray['chunks'])
x_array

array([3445750., 3446750., 3447750., 3448750., 3449750., 3450750.,
       3451750., 3452750., 3453750., 3454750., 3455750., 3456750.,
       3457750., 3458750., 3459750., 3460750., 3461750., 3462750.,
       3463750., 3464750., 3465750., 3466750., 3467750., 3468750.,
       3469750., 3470750., 3471750., 3472750., 3473750., 3474750.,
       3475750., 3476750., 3477750., 3478750., 3479750., 3480750.,
       3481750., 3482750., 3483750., 3484750., 3485750., 3486750.,
       3487750., 3488750., 3489750., 3490750., 3491750., 3492750.,
       3493750., 3494750., 3495750., 3496750., 3497750., 3498750.,
       3499750., 3500750., 3501750., 3502750., 3503750., 3504750.,
       3505750., 3506750., 3507750., 3508750., 3509750., 3510750.,
       3511750., 3512750., 3513750., 3514750., 3515750., 3516750.,
       3517750., 3518750., 3519750., 3520750., 3521750., 3522750.,
       3523750., 3524750., 3525750., 3526750., 3527750., 3528750.,
       3529750., 3530750., 3531750., 3532750., 3533750., 35347

In [83]:
x_array.shape

(364,)

Oh, look at that, it's a one-dimensional array of the cell x coordinates in the projected coordinate system.

Let's look at `y` too.

In [84]:
y_zarray = read_zarr_json('y/.zarray')
dt = np.dtype(y_zarray['dtype'])
y_array = np.frombuffer(read_zarr_blosc('y/0'), dtype=dt).reshape(y_zarray['chunks'])
y_array

array([-1765000., -1766000., -1767000., -1768000., -1769000., -1770000.,
       -1771000., -1772000., -1773000., -1774000., -1775000., -1776000.,
       -1777000., -1778000., -1779000., -1780000., -1781000., -1782000.,
       -1783000., -1784000., -1785000., -1786000., -1787000., -1788000.,
       -1789000., -1790000., -1791000., -1792000., -1793000., -1794000.,
       -1795000., -1796000., -1797000., -1798000., -1799000., -1800000.,
       -1801000., -1802000., -1803000., -1804000., -1805000., -1806000.,
       -1807000., -1808000., -1809000., -1810000., -1811000., -1812000.,
       -1813000., -1814000., -1815000., -1816000., -1817000., -1818000.,
       -1819000., -1820000., -1821000., -1822000., -1823000., -1824000.,
       -1825000., -1826000., -1827000., -1828000., -1829000., -1830000.,
       -1831000., -1832000., -1833000., -1834000., -1835000., -1836000.,
       -1837000., -1838000., -1839000., -1840000., -1841000., -1842000.,
       -1843000., -1844000., -1845000., -1846000., 

In [85]:
y_array.shape

(231,)

`y` is one-dimensional as well. Not only that, but the shapes of each map to the opposing dimensions of our `lat` and `lon` array shapes. Seems like we're on the right track here, if we only could convert our POI into the projected coordinates...

### Understanding our reference system

One outstanding problem with zarr is that the geospatial extension still has yet to be formalized. This means we don't have a built-in way of grabbing the CRS information from the zarr itself. Planetary Computer gets around this issue by storing the CRS information in the STAC collection, specifically as part of the `cube:dimensions` metadata.

Both the `x` and `y` dimensions have a reference system defined. Thankfully for us, the data provide has made a sane decision to ensure these CRSs are the same, so we only need to use one of them to get a transformer we can use to project our POI coordinates.

In [86]:
collection.extra_fields['cube:dimensions']['x']['reference_system']

{'name': 'undefined',
 'type': 'ProjectedCRS',
 '$schema': 'https://proj.org/schemas/v0.4/projjson.schema.json',
 'base_crs': {'name': 'undefined',
  'datum': {'name': 'undefined',
   'type': 'GeodeticReferenceFrame',
   'ellipsoid': {'name': 'undefined',
    'semi_major_axis': 6378137,
    'inverse_flattening': 298.257223563}},
  'coordinate_system': {'axis': [{'name': 'Longitude',
     'unit': 'degree',
     'direction': 'east',
     'abbreviation': 'lon'},
    {'name': 'Latitude',
     'unit': 'degree',
     'direction': 'north',
     'abbreviation': 'lat'}],
   'subtype': 'ellipsoidal'}},
 'conversion': {'name': 'unknown',
  'method': {'id': {'code': 9802, 'authority': 'EPSG'},
   'name': 'Lambert Conic Conformal (2SP)'},
  'parameters': [{'id': {'code': 8823, 'authority': 'EPSG'},
    'name': 'Latitude of 1st standard parallel',
    'unit': 'degree',
    'value': 25},
   {'id': {'code': 8824, 'authority': 'EPSG'},
    'name': 'Latitude of 2nd standard parallel',
    'unit': 'degre

In [87]:
src_crs = pyproj.CRS.from_json_dict(collection.extra_fields['cube:dimensions']['x']['reference_system'])
src_crs

<Projected CRS: {"name": "undefined", "type": "ProjectedCRS", "$sc ...>
Name: undefined
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Lambert Conic Conformal (2SP)
Datum: undefined
- Ellipsoid: undefined
- Prime Meridian: Greenwich

In [88]:
to_src_transformer = pyproj.Transformer.from_crs("EPSG:4326", src_crs)

In [89]:
poi_projected_x, poi_projected_y = to_src_transformer.transform(
    POI.y,
    POI.x,
)
poi_projected_x, poi_projected_y

(3626464.519424091, -1878320.0944692981)

### Calculating our cell coords with a "geotransform"

At this point we're really just in regular raster land and find outselves needing the raster grid's affine transformation--or geotransform in GDAL speak--to convert our coordinates from model space to grid space. Other than having to continue to pull metadata out of the collection to get our inputs, we can calculate the values we need to create that data structure. 

In [90]:
geotransform = (
    collection.extra_fields['cube:dimensions']['x']['extent'][0],
    collection.extra_fields['cube:dimensions']['x']['step'],
    0,
    collection.extra_fields['cube:dimensions']['y']['extent'][1],
    0,
    collection.extra_fields['cube:dimensions']['y']['step'],
)
geotransform

(3445750.0, 1000.0, 0, -1765000.0, 0, -1000.0)

With that, we can finally do the normal conversion of geographic coordinates into grid coordinates.

In [91]:
POI_col = int((poi_projected_x - geotransform[0]) // geotransform[1])
POI_row = int((poi_projected_y - geotransform[3]) // geotransform[5])
POI_row, POI_col

(113, 180)

In [92]:
lon_array[POI_row, POI_col], lat_array[POI_row, POI_col]

(np.float32(-66.06871), np.float32(18.4114))

Does that look right?

### Another option to find the cell coords

Remember that the `x` and `y` arrays are one-dimensional and increasing/decreasing by a constant step? We could have also used that property to perform a linear search of the coordinate space to get our row and column coordinates.

In [93]:
col = 0
POI_row_ = None
while poi_projected_y < y_array[col]:
    POI_row_ = col
    col += 1
POI_row_

113

In [94]:
col = 0
POI_col_ = None
while poi_projected_x >= x_array[col]:
    POI_col_ = col
    col += 1
POI_col_

180

Note that this strategy worked in this case because we had all of our x and all our y coordinate values in single arrays. If we were working with a larger zarr dataset this might not have been the case. Also, this strategy is not particularly efficient. As a general rule it is probably best to stick to the coordinate calculation using the offsets from the grid origin rather than this more brute-force strategy.

## Reading the `tmax` variable data

If after all this you still remember what we are trying to do, great! If not, it's understandable, so here's a little refresher: we are interested in finding the week of 2020 with the lowest average (mean) `tmax` value. So it seems pertinent that we read in the `tmax` variable data.

Let's take a look at what's in the `tmax` directory.

In [95]:
ls_zarr('tmax')

['daymet-zarr/daily/pr.zarr/tmax/.zarray',
 'daymet-zarr/daily/pr.zarr/tmax/.zattrs',
 'daymet-zarr/daily/pr.zarr/tmax/0.0.0',
 'daymet-zarr/daily/pr.zarr/tmax/1.0.0',
 'daymet-zarr/daily/pr.zarr/tmax/10.0.0',
 'daymet-zarr/daily/pr.zarr/tmax/11.0.0',
 'daymet-zarr/daily/pr.zarr/tmax/12.0.0',
 'daymet-zarr/daily/pr.zarr/tmax/13.0.0',
 'daymet-zarr/daily/pr.zarr/tmax/14.0.0',
 'daymet-zarr/daily/pr.zarr/tmax/15.0.0',
 'daymet-zarr/daily/pr.zarr/tmax/16.0.0',
 'daymet-zarr/daily/pr.zarr/tmax/17.0.0',
 'daymet-zarr/daily/pr.zarr/tmax/18.0.0',
 'daymet-zarr/daily/pr.zarr/tmax/19.0.0',
 'daymet-zarr/daily/pr.zarr/tmax/2.0.0',
 'daymet-zarr/daily/pr.zarr/tmax/20.0.0',
 'daymet-zarr/daily/pr.zarr/tmax/21.0.0',
 'daymet-zarr/daily/pr.zarr/tmax/22.0.0',
 'daymet-zarr/daily/pr.zarr/tmax/23.0.0',
 'daymet-zarr/daily/pr.zarr/tmax/24.0.0',
 'daymet-zarr/daily/pr.zarr/tmax/25.0.0',
 'daymet-zarr/daily/pr.zarr/tmax/26.0.0',
 'daymet-zarr/daily/pr.zarr/tmax/27.0.0',
 'daymet-zarr/daily/pr.zarr/tmax/28

Uh oh. This is different than we've run into up till now. We see a bunch of different chunk files here. Maybe if we look at the `.zarray` and `.zattrs` we can figure out something to help us here.

In [96]:
tmax_zarray = read_zarr_json('tmax/.zarray')
print_json(tmax_zarray)

{
    "chunks": [
        365,
        231,
        364
    ],
    "compressor": {
        "blocksize": 0,
        "clevel": 5,
        "cname": "lz4",
        "id": "blosc",
        "shuffle": 1
    },
    "dtype": "<f4",
    "fill_value": "NaN",
    "filters": null,
    "order": "C",
    "shape": [
        14965,
        231,
        364
    ],
    "zarr_format": 2
}


In [97]:
tmax_zattrs = read_zarr_json('tmax/.zattrs')
print_json(tmax_zattrs)

{
    "_ARRAY_DIMENSIONS": [
        "time",
        "y",
        "x"
    ],
    "cell_methods": "area: mean time: maximum",
    "coordinates": "lon lat",
    "grid_mapping": "lambert_conformal_conic",
    "long_name": "daily maximum temperature",
    "units": "degrees C"
}


We see with `tmax` that the chunk size `(365, 231, 364)` is different than the shape `(14965, 231, 364)`. And from `.zattrs` we see the array dimensions listed as `(time, y, x)`. A deductive person could infer that, this being a daily dataset, the chunk having a size of `365` for the `time` dimension means that each chunk is a year's worth of values. We have 41 chunk files (`0.0.0` though `40.0.0`), and `365 * 41 = 14965`. It further stands to reason that the chunks are in chronological order, so we can reasonably assume we want the last chunk, `40.0.0`.

But what good would it do just to be clever here? Instead, let's confirm our theory. To do so we can look at the `time` variable.

In [98]:
ls_zarr('time')

['daymet-zarr/daily/pr.zarr/time/.zarray',
 'daymet-zarr/daily/pr.zarr/time/.zattrs',
 'daymet-zarr/daily/pr.zarr/time/0',
 'daymet-zarr/daily/pr.zarr/time/1',
 'daymet-zarr/daily/pr.zarr/time/10',
 'daymet-zarr/daily/pr.zarr/time/11',
 'daymet-zarr/daily/pr.zarr/time/12',
 'daymet-zarr/daily/pr.zarr/time/13',
 'daymet-zarr/daily/pr.zarr/time/14',
 'daymet-zarr/daily/pr.zarr/time/15',
 'daymet-zarr/daily/pr.zarr/time/16',
 'daymet-zarr/daily/pr.zarr/time/17',
 'daymet-zarr/daily/pr.zarr/time/18',
 'daymet-zarr/daily/pr.zarr/time/19',
 'daymet-zarr/daily/pr.zarr/time/2',
 'daymet-zarr/daily/pr.zarr/time/20',
 'daymet-zarr/daily/pr.zarr/time/21',
 'daymet-zarr/daily/pr.zarr/time/22',
 'daymet-zarr/daily/pr.zarr/time/23',
 'daymet-zarr/daily/pr.zarr/time/24',
 'daymet-zarr/daily/pr.zarr/time/25',
 'daymet-zarr/daily/pr.zarr/time/26',
 'daymet-zarr/daily/pr.zarr/time/27',
 'daymet-zarr/daily/pr.zarr/time/28',
 'daymet-zarr/daily/pr.zarr/time/29',
 'daymet-zarr/daily/pr.zarr/time/3',
 'daym

In [99]:
time_zarray = read_zarr_json('time/.zarray')
print_json(time_zarray)

{
    "chunks": [
        365
    ],
    "compressor": {
        "blocksize": 0,
        "clevel": 5,
        "cname": "lz4",
        "id": "blosc",
        "shuffle": 1
    },
    "dtype": "<i8",
    "fill_value": null,
    "filters": null,
    "order": "C",
    "shape": [
        14965
    ],
    "zarr_format": 2
}


In [101]:
suspected_year_chunk = 2020-1980
suspected_year_chunk

40

In [108]:
dt = np.dtype(time_zarray['dtype'])
time_array = np.frombuffer(read_zarr_blosc(f'time/{suspected_year_chunk}'), dtype=dt).reshape(time_zarray['chunks'])
time_array

array([14610, 14611, 14612, 14613, 14614, 14615, 14616, 14617, 14618,
       14619, 14620, 14621, 14622, 14623, 14624, 14625, 14626, 14627,
       14628, 14629, 14630, 14631, 14632, 14633, 14634, 14635, 14636,
       14637, 14638, 14639, 14640, 14641, 14642, 14643, 14644, 14645,
       14646, 14647, 14648, 14649, 14650, 14651, 14652, 14653, 14654,
       14655, 14656, 14657, 14658, 14659, 14660, 14661, 14662, 14663,
       14664, 14665, 14666, 14667, 14668, 14669, 14670, 14671, 14672,
       14673, 14674, 14675, 14676, 14677, 14678, 14679, 14680, 14681,
       14682, 14683, 14684, 14685, 14686, 14687, 14688, 14689, 14690,
       14691, 14692, 14693, 14694, 14695, 14696, 14697, 14698, 14699,
       14700, 14701, 14702, 14703, 14704, 14705, 14706, 14707, 14708,
       14709, 14710, 14711, 14712, 14713, 14714, 14715, 14716, 14717,
       14718, 14719, 14720, 14721, 14722, 14723, 14724, 14725, 14726,
       14727, 14728, 14729, 14730, 14731, 14732, 14733, 14734, 14735,
       14736, 14737,

Per the metadata for the `time` array, we can interpret these values as the date of that cell in the days since 1980-01-01. We can use the `datetime` module to calculate the date given by cell 0 in the array above:

In [110]:
# to confirm the dates match up
chunk_start_date = datetime.date.fromisoformat('19800101') + datetime.timedelta(days=int(time_array[0]))
print(chunk_start_date)

2020-01-01


Ah, great, proof that chunk 40 is the chunk we want for 2020 values.

Now that we know which chunk we want, let's finally read it. Note that this array is significantly bigger--365 times bigger, to be exact--than our `lat` and `lon` arrays. As a result, it might take a bit longer to download, extract, and unpack into an array. Be patient. In testing this typically took on the order of 30-40 seconds.

In [111]:
dt = np.dtype(tmax_zarray['dtype'])
tmax_array = np.frombuffer(read_zarr_blosc(f'tmax/{suspected_year_chunk}.0.0'), dtype=dt).reshape(tmax_zarray['chunks'])
tmax_array.shape

(365, 231, 364)

What were we trying to do with these values again? Oh yeah, find the average max temperature of each week in the year, so we could identify the minimum. We can do this pretty trivially with `numpy` by:

* slicing out only the stack of temperature values we're interested in for our POI cell
* reshaping the array into rows of seven values
* finding the mean along the row axis (gives us the mean of each week)
* find the min value within that weekly mean array

In [112]:
# we only take 364 temperature values because 365 isn't divisible by 7 😕
POI_tmax_array = tmax_array[:364, POI_row, POI_col].reshape(52, 7)
POI_tmax_array

array([[29.42, 29.16, 27.63, 28.53, 28.47, 27.91, 28.76],
       [29.19, 28.79, 28.74, 28.95, 28.26, 28.  , 27.44],
       [28.03, 28.29, 28.51, 28.1 , 27.33, 28.26, 30.1 ],
       [29.08, 29.95, 28.13, 28.49, 28.31, 28.96, 29.42],
       [29.41, 30.58, 30.1 , 29.66, 30.36, 30.14, 29.76],
       [29.57, 30.64, 30.29, 29.46, 29.28, 30.25, 29.58],
       [28.81, 29.5 , 28.83, 28.55, 28.33, 28.95, 29.23],
       [28.52, 29.21, 29.31, 30.71, 30.27, 26.03, 26.17],
       [28.02, 28.89, 29.16, 29.88, 29.39, 27.94, 28.6 ],
       [29.  , 29.01, 28.89, 29.58, 28.52, 26.28, 28.29],
       [27.78, 30.69, 29.52, 28.29, 27.75, 27.41, 27.79],
       [27.46, 27.94, 28.34, 27.97, 27.73, 27.22, 27.14],
       [28.47, 29.27, 28.85, 28.93, 29.01, 27.99, 28.95],
       [31.28, 31.6 , 30.85, 31.05, 30.02, 30.03, 28.52],
       [27.81, 30.34, 32.13, 31.26, 31.38, 30.11, 30.1 ],
       [30.24, 30.28, 30.18, 30.13, 31.16, 31.34, 32.29],
       [31.8 , 30.91, 31.04, 31.06, 31.49, 31.27, 31.26],
       [30.81,

In [113]:
POI_tmax_mean_array = np.mean(POI_tmax_array, axis=1)
POI_tmax_mean_array

array([28.554285, 28.481428, 28.374285, 28.905712, 30.001429, 29.867144,
       28.885714, 28.602858, 28.84    , 28.51    , 28.46143 , 27.685715,
       28.78143 , 30.478573, 30.447144, 30.802856, 31.261429, 31.307142,
       32.63    , 32.295715, 32.897144, 32.164284, 32.04143 , 32.354286,
       32.495712, 32.092857, 32.567142, 32.30143 , 31.725714, 31.554287,
       31.612856, 31.941427, 31.832855, 31.319998, 31.648571, 31.817142,
       31.908571, 32.287144, 31.97143 , 30.872858, 31.844284, 31.012857,
       32.142857, 30.960001, 30.135712, 29.641428, 30.059998, 29.119999,
       28.982859, 29.215715, 29.369999, 28.734285], dtype=float32)

In [117]:
POI_tmax_min = np.min(POI_tmax_mean_array)
POI_tmax_min_index = np.argmin(POI_tmax_mean_array)
print(f'The lowest tmax temp {POI_tmax_min} occurred in week {POI_tmax_min_index} of the 2020.')

The lowest tmax temp 27.685714721679688 occurred in week 11 of the 2020.


This is a great result! But it would be good if we could turn that into an actual date. Good thing we read in that `time` array: we can use slice it on weekly bounds to get the date of the first day of the week in question.

In [118]:
week__days_since_1980 = int(time_array[::7][POI_tmax_min_index])
week__days_since_1980

14687

In [119]:
best_week_of_the_year_start = datetime.date.fromisoformat('19800101') + datetime.timedelta(days=week__days_since_1980)
print(f'The best day to arrive in Puerto Rico, from our data, was {best_week_of_the_year_start}. Now go book your time machine tickets. 😁')

The best day to arrive in Puerto Rico, from our data, was 2020-03-18. Now go book your time machine tickets. 😁
